## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


In [ ]:
# Create a list of at least 10 column names to use as X data
x = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked'] 

# Create X_df using your selected columns
X_df = attrition_df[x]

# Show the data types for X_df
X_df.dtypes


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

train_overtime = X_train['OverTime']
test_overtime = X_test['OverTime']

le = LabelEncoder()
train_overtime_encoded = le.fit_transform(train_overtime)
test_overtime_encoded = le.transform(test_overtime)

X_train['OverTime'] = train_overtime_encoded
X_test['OverTime'] = test_overtime_encoded

In [ ]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled


In [ ]:
# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder()

# Fit the encoder to the training data
ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dep_encoded = ohe.transform(y_train[['Department']]).toarray()
y_test_dep_encoded = ohe.transform(y_test[['Department']]).toarray()
y_train_dep_encoded

In [ ]:
# Create a OneHotEncoder for the Attrition column
ohe = OneHotEncoder()

# Fit the encoder to the training data
ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoded = ohe.transform(y_train[['Attrition']]).toarray()
y_test_attr_encoded = ohe.transform(y_test[['Attrition']]).toarray()
y_train_attr_encoded


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
X_column_length = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_column_length,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dep_hidden_layer = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(dep_hidden_layer)


In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden_layer = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name = 'attrition_output')(attr_hidden_layer)


In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output':'accuracy',
                       'attrition_output':'accuracy'}
                    )

# Summarize the model
model.summary()

In [ ]:
# Train the model
model.fit(X_train_scaled,
          {'department_output':y_train_dep_encoded, 'attrition_output': y_train_attr_encoded},
          epochs=100,
          batch_size=32,
          validation_split=0.2
          )


In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled,
                              {'department_output':y_test_dep_encoded, 'attrition_output': y_test_attr_encoded})

In [ ]:
# Print the accuracy for both department and attrition
test_results

In [ ]:
attrition_df['Attrition'].value_counts()

In [ ]:
attrition_df['Department'].value_counts()

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric for evaluating models predicting employee attrition and departmental fit due to potential class imbalance, where a model could achieve high accuracy by predicting the majority class. Additionally, a false negative in attrition prediction (failing to predict that an employee will leave) could have more serious consequences than a false positive (predicting that an employee will leave when they do not). 
According to the Xpert Learning Assistant, metrics like precision, recall, F1-score, and AUC-ROC can provide more valuable insights, especially in imbalanced datasets. Recall measures how well the model identifies employees likely to leave, while precision assesses the accuracy of those predictions. AUC-ROC evaluates the model's performance across different thresholds, considering both true and false positives.

2. For employee attrition prediction, I used the sigmoid activation function as it is a binary classification task, which outputs a probability between 0 and 1 for whether an employee will leave or stay. For department prediction, I used the softmax activation function as it is a multi-class classification task, which outputs a probability distribution across multiple departments, with the highest probability indicating the best fit for the employee.

3. Changing the loss function for each output to account for the imbalanced data. Get more data to balance the counts for the classes in attrition and departments.